In [1]:
!git init .
!git remote add origin https://github.com/kashparty/STP-GSR.git
!git pull origin discriminator

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 127 (delta 10), reused 8 (delta 4), pack-reused 102 (from 1)
Receiving objects: 100% (127/127), 64.80 MiB | 3.98 MiB/s, done.
Resolving deltas: 100% (53/53), done.
From https://github.com/kashparty/STP-GSR
 * branch            discriminator -> FETCH_HEAD
 * [new branch]      discriminator -> origin/discriminator
U

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
import os
import hydra
import torch
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import KFold
from hydra import compose, initialize

from src.train import train, eval
from src.plot_utils import plot_adj_matrices
from src.dataset import load_dataset

def main():
    with initialize(version_base=None, config_path="configs"):
        config = compose(config_name="experiment")

    torch.cuda.empty_cache()

    if torch.cuda.is_available():
        print("Running on GPU")
    else:
        print("Running on CPU")

    kf = KFold(n_splits=config.experiment.kfold.n_splits,
               shuffle=config.experiment.kfold.shuffle,
               random_state=config.experiment.kfold.random_state)

    # Initialize folder structure for this run
    base_dir = config.experiment.base_dir
    model_name = config.model.name
    dataset_type = config.dataset.name
    run_name = config.experiment.run_name
    run_dir = f'{base_dir}/{model_name}/{dataset_type}/{run_name}/'

    # Load dataset
    source_data, target_data = load_dataset(config)

    for fold, (train_idx, val_idx) in enumerate(kf.split(source_data)):
        print(f"Training Fold {fold+1}/3")

        # Initialize results directory
        res_dir = f'{run_dir}fold_{fold+1}/'
        if not os.path.exists(res_dir):
            os.makedirs(res_dir)

        # Fetch training and val data for this fold
        source_data_train = [source_data[i] for i in train_idx]
        target_data_train = [target_data[i] for i in train_idx]
        source_data_val = [source_data[i] for i in val_idx]
        target_data_val = [target_data[i] for i in val_idx]

        # Train model for this fold
        train_output = train(config,
                              source_data_train,
                              target_data_train,
                              source_data_val,
                              target_data_val,
                              res_dir)

        # Evaluate model for this fold
        eval_output, eval_loss = eval(config,
                                      train_output['model'],
                                      source_data_val,
                                      target_data_val,
                                      train_output['critereon'])

        # Final evaluation loss for this fold
        print(f"Final Validation Loss (Target): {eval_loss}")

        # Save source, taregt, and eval output for this fold
        np.save(f'{res_dir}/eval_output.npy', np.array(eval_output))
        np.save(f'{res_dir}/source.npy', np.array([s['mat'] for s in source_data_val]))
        np.save(f'{res_dir}/target.npy', np.array([t['mat'] for t in target_data_val]))


        # Plot predictions for a random sample
        idx = 6
        source_mat_test = source_data_val[idx]['mat']
        target_mat_test = target_data_val[idx]['mat']
        eval_output_t = eval_output[idx]

        plot_adj_matrices(source_mat_test,
                          target_mat_test,
                          eval_output_t,
                          idx,
                          res_dir,
                          file_name=f'eval_sample{idx}')


main()

Running on CPU
Training Fold 1/3


  0%|          | 0/111 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (268x268 and 320x160)

In [1]:
from hydra import compose, initialize
from src.models.stp_gsr import STPGSR
import torch

with initialize(version_base=None, config_path="configs"):
    config = compose(config_name="experiment")

model = STPGSR(config)
model.load_state_dict(torch.load("model.pth", map_location=torch.device("cpu")))

<All keys matched successfully>

In [2]:
from src.matrix_vectorizer import MatrixVectorizer
from src.dataset import create_pyg_graph
from functools import partial
import numpy as np

source_vectorized = np.genfromtxt("lr_test.csv", delimiter=",", skip_header=1)
source_mat_all = [MatrixVectorizer.anti_vectorize(A, 160) for A in source_vectorized]

source_mat_all = [torch.tensor(x, dtype=torch.float) for x in source_mat_all]
pyg_partial = partial(create_pyg_graph, node_feature_init="adj", node_feat_dim=160)

source_pyg_all = [pyg_partial(x, 160) for x in source_mat_all]
source_data = [{'pyg': source_pyg, 'mat': source_mat} for source_pyg, source_mat in zip(source_pyg_all, source_mat_all)]

In [3]:
from src.dual_graph_utils import revert_dual

model.eval()
eval_output = []

with torch.no_grad():
    for source in source_data:
        source_g = source['pyg']

        model_pred, model_target, _, _ = model(source_g, None)
        pred_m = revert_dual(model_pred, 268)    # (n_t, n_t)
        pred_m = pred_m.cpu().numpy()
        eval_output.append(pred_m)

eval_output

[array([[0.        , 0.44771627, 0.55845326, ..., 0.26248917, 0.21184392,
         0.16114874],
        [0.44771627, 0.        , 0.70342153, ..., 0.27266598, 0.26790196,
         0.23088662],
        [0.55845326, 0.70342153, 0.        , ..., 0.3551074 , 0.3569528 ,
         0.33785626],
        ...,
        [0.26248917, 0.27266598, 0.3551074 , ..., 0.        , 0.48278287,
         0.39487475],
        [0.21184392, 0.26790196, 0.3569528 , ..., 0.48278287, 0.        ,
         0.4911103 ],
        [0.16114874, 0.23088662, 0.33785626, ..., 0.39487475, 0.4911103 ,
         0.        ]], dtype=float32),
 array([[0.        , 0.44666672, 0.6168959 , ..., 0.        , 0.        ,
         0.        ],
        [0.44666672, 0.        , 0.82750386, ..., 0.        , 0.04713491,
         0.        ],
        [0.6168959 , 0.82750386, 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.2947315 ,
         0.37615126

In [22]:
from src.matrix_vectorizer import MatrixVectorizer
import pandas as pd

test_array = np.concatenate([MatrixVectorizer.vectorize(eo) for eo in eval_output])

output_df = pd.DataFrame({"Predicted": test_array.flatten()})
output_df.index = np.arange(1, len(output_df) + 1)
output_df.to_csv("submission.csv", index_label="ID")
output_df

,Predicted
1,0.447716
2,0.558453
3,0.703422
4,0.523096
5,0.359028
...,...
4007132,0.026071
4007133,0.000000
4007134,0.014338
4007135,0.300120
